# Setup

## PAckage Imports

In [4]:
import pandas as pd
import numpy as np
import requests
import regex as re
import time
import os
import gc

# ignore warnings (gets rid of Pandas copy warnings)
import warnings
warnings.filterwarnings('ignore')
pd.options.display.max_columns = None

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 30)

# NLP tools
import spacy
nlp = spacy.load("en_core_web_sm")
import re
import nltk
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from nltk.tokenize import word_tokenize

from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import os

## Notebook Functions

In [42]:
users = ['mexican_moon_BGG', 'mexicojramirez', 'mexicola']

In [46]:
user_path = ""

for item in users:
    user = re.sub("\s+", "+", item)
    user_path += user+','

path = "https://www.boardgamegeek.com/xmlapi2/collection?username="+user_path+"&rated=1&stats=1"
start = time.time()# log the start time for this entry
    
print(path)
    
flag = False


https://www.boardgamegeek.com/xmlapi2/collection?username=mexican_moon_BGG,mexicojramirez,mexicola,&rated=1&stats=1


In [ ]:
'''
while flag == False:
        
    print("Retrieving page")
        
    page = requests.get(path) # get the page
    game_page = BeautifulSoup(page.content, "xml") # parse the page with beautifulsoup        
                
    if game_page.find('errors') != None:
        print("Invalid username")
        break
        
    else: 
        try:
            print(int(game_page.find('items')['totalitems']))
            flag = True
            continue
        except:
            print("failed, pausing")
            time.sleep(1)
            continue'''

In [5]:
def get_user_ratings(username):
    
    user_path = re.sub("\s+", "+", username)
    
    path = "https://www.boardgamegeek.com/xmlapi2/collection?username="+user_path+"&rated=1&stats=1"
    start = time.time()# log the start time for this entry
    
    print(path)
    
    flag = False
    
    while flag == False:
        
        print("Retrieving page")
        
        page = requests.get(path) # get the page
        game_page = BeautifulSoup(page.content, "xml") # parse the page with beautifulsoup        
                
        if game_page.find('errors') != None:
            print("Invalid username")
            break
        
        else: 
            try:
                print(int(game_page.find('items')['totalitems']))
                flag = True
                continue
            except:
                print("failed, pausing")
                time.sleep(1)
                continue
           
    rated_items = game_page.find_all("item")
        
    game_ids = []
    game_ratings = []
    modified_record = []
    all_ratings = {}

    for game in rated_items:
        name = game.find("name").text
        game_id = game["objectid"] 
        rating = float(game.find("rating")["value"])
        date_rated = game.find("status")["lastmodified"]
        game_ids.append(game_id)
        game_ratings.append(rating)
        all_ratings[game_id]=rating
        modified = [username, game_id, rating, date_rated]
        modified_record.append(modified)

    raw_ratings_dict = {}
    raw_ratings_dict[username] = all_ratings
    
    user = pd.DataFrame.from_dict(raw_ratings_dict, orient="index")
    user_ratings_dates = pd.DataFrame(modified_record, columns=["Username", "BGGId", "Rating", "Date_rated"])
    
    time.sleep(.5)        
    
    return user, user_ratings_dates


In [6]:
def save_load_loop(users):
    
    ratings_dates = pd.read_pickle('userid/ratings_dates.pkl')
    user_ratings = pd.read_pickle('userid/user_ratings.pkl')

    print(users[:])
    
    for username in users:
    
        user, user_ratings_dates = get_user_ratings(username)
    
        ratings_dates = ratings_dates.append(user_ratings_dates)
        user_ratings = user_ratings.append(user)
        
        del user
        del user_ratings_dates
        gc.collect()
    
    float_conversion = user_ratings.astype('float')
    
    ratings_dates.to_pickle('userid/ratings_dates.pkl')
    float_conversion.to_pickle('userid/user_ratings.pkl')
    
    del ratings_dates
    del user_ratings
    gc.collect()

In [6]:
def save_load_loop(users):
    
    ratings_dates = pd.read_pickle('userid/ratings_dates.pkl')
    user_ratings = pd.read_pickle('userid/user_ratings.pkl')

    print(users[:])
    
    for username in users:
    
        user, user_ratings_dates = get_user_ratings(username)
    
        ratings_dates = ratings_dates.append(user_ratings_dates)
        user_ratings = user_ratings.append(user)
        
        del user
        del user_ratings_dates
        gc.collect()
    
    float_conversion = user_ratings.astype('float')
    
    ratings_dates.to_pickle('userid/ratings_dates.pkl')
    float_conversion.to_pickle('userid/user_ratings.pkl')
    
    del ratings_dates
    del user_ratings
    gc.collect()

## Loading Comments

In [7]:
comments = pd.read_pickle('data_cleaned/comments.pkl')
comments

,BGGId,Name,Rating,Username,cleaned
0,174430,Gloomhaven,N/A,-Johnny-,con fiddliness time investment ridiculous mo...
1,174430,Gloomhaven,9,-mIDE-,kickstarter estimate delivery
2,174430,Gloomhaven,N/A,0 1 1 2 3 5 8,half good forge war
3,174430,Gloomhaven,10,0stuart0,good addictive coop game ve play desperately p...
4,174430,Gloomhaven,5,1 Family Meeple,imghttpscfgeekdostaticcommbsmbpngimg gloomh...
...,...,...,...,...,...
4172603,339592,Sheep in Disguise,10,Pitersenpai,este es el primer proyecto que patrocino en ki...
4172604,339592,Sheep in Disguise,10,Radalict,game look lot fun m look forward
4172605,339592,Sheep in Disguise,10,Rigoberto123,nt wait play support kickstarter subscribe mr ...
4172606,339592,Sheep in Disguise,10,TAB4two,wait game hour hilarious sheepnanigan family b...


In [8]:
all_users = sorted(list(comments['Username'].unique()))

del comments
gc.collect()

6363

In [9]:
len(all_users)

182354

In [90]:
all_users.index('zath')

181750

## Get Ratings

In [93]:
sub_users = all_users[181750:]

In [94]:
start = 0

while True:
    
    end = start + 50
    users = sub_users[start:end]
    if len(users)==0: break
    save_load_loop(users)
    start += 50
    time.sleep(3)

['zath', 'zatks', 'zatoichijuegos', 'zaubererer', 'zauberwurf', 'zauis', 'zautman', 'zavel8', 'zavesky', 'zavijahbeta', 'zavulon', 'zawimawi', 'zax', 'zaxa', 'zazasLT', 'zazenpanda', 'zazery', 'zazzman', 'zb1035', 'zb2112', 'zbajszek', 'zberkoff', 'zbgamer4545', 'zbizek1', 'zboat', 'zbone', 'zbrent716', 'zbruss', 'zbunc17', 'zbv500', 'zbyshekh', 'zbyszeklarwa', 'zc1993', 'zcantin', 'zcapr17', 'zcelloman', 'zchillman', 'zco2000', 'zd2046', 'zdalekacelny', 'zdan89', 'zdenda', 'zdepthcharge', 'zdkat', 'zdpierson', 'zdravstvouite', 'zdriesen', 'zdtorok', 'ze_dutch_kiwi', 'ze_stom']
https://www.boardgamegeek.com/xmlapi2/collection?username=zath&rated=1&stats=1
Retrieving page
80
https://www.boardgamegeek.com/xmlapi2/collection?username=zatks&rated=1&stats=1
Retrieving page
19
https://www.boardgamegeek.com/xmlapi2/collection?username=zatoichijuegos&rated=1&stats=1
Retrieving page
25
https://www.boardgamegeek.com/xmlapi2/collection?username=zaubererer&rated=1&stats=1
Retrieving page
1092
http

failed, pausing
Retrieving page
1
https://www.boardgamegeek.com/xmlapi2/collection?username=zeaklous&rated=1&stats=1
Retrieving page
failed, pausing
Retrieving page
36
https://www.boardgamegeek.com/xmlapi2/collection?username=zeaksevers&rated=1&stats=1
Retrieving page
failed, pausing
Retrieving page
5
https://www.boardgamegeek.com/xmlapi2/collection?username=zeal&rated=1&stats=1
Retrieving page
failed, pausing
Retrieving page
48
https://www.boardgamegeek.com/xmlapi2/collection?username=zealisee&rated=1&stats=1
Retrieving page
failed, pausing
Retrieving page
16
https://www.boardgamegeek.com/xmlapi2/collection?username=zeamadevarza&rated=1&stats=1
Retrieving page
failed, pausing
Retrieving page
failed, pausing
Retrieving page
163
https://www.boardgamegeek.com/xmlapi2/collection?username=zeanarchy&rated=1&stats=1
Retrieving page
failed, pausing
Retrieving page
13
https://www.boardgamegeek.com/xmlapi2/collection?username=zebani200&rated=1&stats=1
Retrieving page
failed, pausing
Retrieving 

failed, pausing
Retrieving page
158
https://www.boardgamegeek.com/xmlapi2/collection?username=zeldaar&rated=1&stats=1
Retrieving page
failed, pausing
Retrieving page
35
https://www.boardgamegeek.com/xmlapi2/collection?username=zeldafire&rated=1&stats=1
Retrieving page
failed, pausing
Retrieving page
78
https://www.boardgamegeek.com/xmlapi2/collection?username=zelekangela&rated=1&stats=1
Retrieving page
failed, pausing
Retrieving page
64
https://www.boardgamegeek.com/xmlapi2/collection?username=zelekar&rated=1&stats=1
Retrieving page
failed, pausing
Retrieving page
32
https://www.boardgamegeek.com/xmlapi2/collection?username=zelig2&rated=1&stats=1
Retrieving page
failed, pausing
Retrieving page
35
https://www.boardgamegeek.com/xmlapi2/collection?username=zelitrex36&rated=1&stats=1
Retrieving page
failed, pausing
Retrieving page
15
https://www.boardgamegeek.com/xmlapi2/collection?username=zeljkov020&rated=1&stats=1
Retrieving page
failed, pausing
Retrieving page
46
https://www.boardgameg

failed, pausing
Retrieving page
1
https://www.boardgamegeek.com/xmlapi2/collection?username=zenwired&rated=1&stats=1
Retrieving page
failed, pausing
Retrieving page
86
https://www.boardgamegeek.com/xmlapi2/collection?username=zenxacred&rated=1&stats=1
Retrieving page
failed, pausing
Retrieving page
126
https://www.boardgamegeek.com/xmlapi2/collection?username=zenz&rated=1&stats=1
Retrieving page
failed, pausing
Retrieving page
62
https://www.boardgamegeek.com/xmlapi2/collection?username=zeotter&rated=1&stats=1
Retrieving page
failed, pausing
Retrieving page
failed, pausing
Retrieving page
failed, pausing
Retrieving page
failed, pausing
Retrieving page
failed, pausing
Retrieving page
failed, pausing
Retrieving page
failed, pausing
Retrieving page
failed, pausing
Retrieving page
failed, pausing
Retrieving page
failed, pausing
Retrieving page
failed, pausing
Retrieving page
failed, pausing
Retrieving page
failed, pausing
Retrieving page
failed, pausing
Retrieving page
failed, pausing
Retr

failed, pausing
Retrieving page
0
https://www.boardgamegeek.com/xmlapi2/collection?username=zeropointfield&rated=1&stats=1
Retrieving page
failed, pausing
Retrieving page
5
https://www.boardgamegeek.com/xmlapi2/collection?username=zerorez&rated=1&stats=1
Retrieving page
failed, pausing
Retrieving page
30
https://www.boardgamegeek.com/xmlapi2/collection?username=zerosatanmine&rated=1&stats=1
Retrieving page
failed, pausing
Retrieving page
47
https://www.boardgamegeek.com/xmlapi2/collection?username=zerostasis&rated=1&stats=1
Retrieving page
failed, pausing
Retrieving page
8
https://www.boardgamegeek.com/xmlapi2/collection?username=zeroth+hour&rated=1&stats=1
Retrieving page
failed, pausing
Retrieving page
125
https://www.boardgamegeek.com/xmlapi2/collection?username=zerothefool&rated=1&stats=1
Retrieving page
failed, pausing
Retrieving page
failed, pausing
Retrieving page
236
https://www.boardgamegeek.com/xmlapi2/collection?username=zerstorer88&rated=1&stats=1
Retrieving page
failed, pa

...

Retrieving page
failed, pausing
Retrieving page
222
https://www.boardgamegeek.com/xmlapi2/collection?username=zkl2003&rated=1&stats=1
Retrieving page
failed, pausing
Retrieving page
31
https://www.boardgamegeek.com/xmlapi2/collection?username=zkolodin&rated=1&stats=1
Retrieving page
failed, pausing
Retrieving page
9
https://www.boardgamegeek.com/xmlapi2/collection?username=zkovac65&rated=1&stats=1
Retrieving page
failed, pausing
Retrieving page
56
https://www.boardgamegeek.com/xmlapi2/collection?username=zkress&rated=1&stats=1
Retrieving page
failed, pausing
Retrieving page
1
https://www.boardgamegeek.com/xmlapi2/collection?username=zlandine&rated=1&stats=1
Retrieving page
failed, pausing
Retrieving page
36
https://www.boardgamegeek.com/xmlapi2/collection?username=zlnemeth&rated=1&stats=1
Retrieving page
failed, pausing
Retrieving page
117
https://www.boardgamegeek.com/xmlapi2/collection?username=zloki83&rated=1&stats=1
Retrieving page
failed, pausing
Retrieving page
18
https://www.boa

failed, pausing
Retrieving page
16
https://www.boardgamegeek.com/xmlapi2/collection?username=zokvicds&rated=1&stats=1
Retrieving page
failed, pausing
Retrieving page
1
https://www.boardgamegeek.com/xmlapi2/collection?username=zol72&rated=1&stats=1
Retrieving page
failed, pausing
Retrieving page
44
https://www.boardgamegeek.com/xmlapi2/collection?username=zolarasec&rated=1&stats=1
Retrieving page
failed, pausing
Retrieving page
128
https://www.boardgamegeek.com/xmlapi2/collection?username=zolarzardoz&rated=1&stats=1
Retrieving page
failed, pausing
Retrieving page
1
https://www.boardgamegeek.com/xmlapi2/collection?username=zoldiz&rated=1&stats=1
Retrieving page
failed, pausing
Retrieving page
136
https://www.boardgamegeek.com/xmlapi2/collection?username=zoliak&rated=1&stats=1
Retrieving page
failed, pausing
Retrieving page
55
https://www.boardgamegeek.com/xmlapi2/collection?username=zolle001&rated=1&stats=1
Retrieving page
failed, pausing
Retrieving page
failed, pausing
Retrieving page
2

failed, pausing
Retrieving page
0
https://www.boardgamegeek.com/xmlapi2/collection?username=zoojar&rated=1&stats=1
Retrieving page
failed, pausing
Retrieving page
17
https://www.boardgamegeek.com/xmlapi2/collection?username=zooko&rated=1&stats=1
Retrieving page
failed, pausing
Retrieving page
10
https://www.boardgamegeek.com/xmlapi2/collection?username=zoomom&rated=1&stats=1
Retrieving page
failed, pausing
Retrieving page
1
https://www.boardgamegeek.com/xmlapi2/collection?username=zooney&rated=1&stats=1
Retrieving page
failed, pausing
Retrieving page
22
https://www.boardgamegeek.com/xmlapi2/collection?username=zoooch25&rated=1&stats=1
Retrieving page
failed, pausing
Retrieving page
152
https://www.boardgamegeek.com/xmlapi2/collection?username=zoop&rated=1&stats=1
Retrieving page
failed, pausing
Retrieving page
failed, pausing
Retrieving page
272
https://www.boardgamegeek.com/xmlapi2/collection?username=zooradio&rated=1&stats=1
Retrieving page
failed, pausing
Retrieving page
3
https://w

failed, pausing
Retrieving page
1
https://www.boardgamegeek.com/xmlapi2/collection?username=zsonody&rated=1&stats=1
Retrieving page
failed, pausing
Retrieving page
0
https://www.boardgamegeek.com/xmlapi2/collection?username=zstoltz&rated=1&stats=1
Retrieving page
failed, pausing
Retrieving page
61
https://www.boardgamegeek.com/xmlapi2/collection?username=zsusyt&rated=1&stats=1
Retrieving page
failed, pausing
Retrieving page
51
https://www.boardgamegeek.com/xmlapi2/collection?username=zsuzs&rated=1&stats=1
Retrieving page
failed, pausing
Retrieving page
17
https://www.boardgamegeek.com/xmlapi2/collection?username=ztfdrake61&rated=1&stats=1
Retrieving page
failed, pausing
Retrieving page
13
https://www.boardgamegeek.com/xmlapi2/collection?username=ztilleto&rated=1&stats=1
Retrieving page
failed, pausing
Retrieving page
109
https://www.boardgamegeek.com/xmlapi2/collection?username=ztimmons&rated=1&stats=1
Retrieving page
failed, pausing
Retrieving page
39
https://www.boardgamegeek.com/xml

Retrieving page
failed, pausing
Retrieving page
failed, pausing
Retrieving page
failed, pausing
Retrieving page
failed, pausing
Retrieving page
failed, pausing
Retrieving page
failed, pausing
Retrieving page
failed, pausing
Retrieving page
failed, pausing
Retrieving page
1057
https://www.boardgamegeek.com/xmlapi2/collection?username=zvsmith&rated=1&stats=1
Retrieving page
failed, pausing
Retrieving page
2
https://www.boardgamegeek.com/xmlapi2/collection?username=zwafke&rated=1&stats=1
Retrieving page
failed, pausing
Retrieving page
0
https://www.boardgamegeek.com/xmlapi2/collection?username=zwalex&rated=1&stats=1
Retrieving page
failed, pausing
Retrieving page
failed, pausing
Retrieving page
217
https://www.boardgamegeek.com/xmlapi2/collection?username=zwarbo&rated=1&stats=1
Retrieving page
failed, pausing
Retrieving page
66
https://www.boardgamegeek.com/xmlapi2/collection?username=zwbsch&rated=1&stats=1
Retrieving page
failed, pausing
Retrieving page
26
https://www.boardgamegeek.com/x

In [ ]:
#ratings_dates = pd.read_pickle('userid/ratings_dates40.pkl')
user_ratings = pd.read_pickle('userid/user_ratings.pkl')

In [ ]:
user_ratings

In [ ]:
ratings_dates

In [ ]:
break

##### ONLY RUN THIS TO RESET THE PICKLES ENTIRELY

In [91]:
ratings_dates = pd.DataFrame(columns=['Username', 'BGGId', 'Rating', 'Date_rated'])

user_ratings = pd.DataFrame()

In [92]:
ratings_dates.to_pickle('userid/ratings_dates.pkl')
user_ratings.to_pickle('userid/user_ratings.pkl')

## Code Work

In [ ]:
break

In [ ]:
username = 'Shade92008'

In [ ]:
path = "https://www.boardgamegeek.com/xmlapi2/collection?username=Shade92008&rated=1&stats=1"

In [ ]:
path = "https://www.boardgamegeek.com/xmlapi2/collection?username=AK&rated=1&stats=1"

In [ ]:
start = time.time()# log the start time for this entry

print(path)
    
page = requests.get(path) # get the page
game_page = BeautifulSoup(page.content, 'xml') # parse the page with beautifulsoup


In [ ]:
game_page.find('errors') != None

In [ ]:
game_page.find('message').text == 'Invalid username specified'

In [ ]:
try: 
    game_page.find('message').text == 'Invalid username specified'
    print("is true")
except: pass

In [ ]:
int(game_page.find('items')['totalitems']) > 0

In [ ]:
rated_items = game_page.find_all('item')
rated_items

In [ ]:
game_ids = []
game_ratings = []
modified_record = []
all_ratings = {}

for game in rated_items:
    name = game.find('name').text
    game_id = game['objectid'] 
    rating = game.find('rating')['value']
    date_rated = game.find('status')['lastmodified']
    print(game_id, rating, date_rated)
    game_ids.append(game_id)
    game_ratings.append(rating)
    all_ratings[game_id]=rating
    modified = [username, game_id, rating, date_rated]
    modified_record.append(modified)

raw_ratings_dict = {}


In [ ]:
modified_record

In [ ]:
raw_ratings_dict[username] = all_ratings
raw_ratings_dict

In [ ]:
user = pd.DataFrame.from_dict(raw_ratings_dict, orient='index')
user

In [ ]:
user_ratings_dates_test = pd.DataFrame(modified_record, columns=['Username', 'BGGId', 'Rating', 'Date_rated'])
user_ratings_dates_test

In [ ]:
ratings_dates_test = pd.DataFrame(columns=['Username', 'BGGId', 'Rating', 'Date_rated'])
ratings_dates_test

In [ ]:
ratings_dates_test = ratings_dates_test.append(user_ratings_dates_test)
ratings_dates_test

In [ ]:
def get_user_ratings(username):
    
    user_path = re.sub("\s+", "+", username)
    
    path = "https://www.boardgamegeek.com/xmlapi2/collection?username="+user_path+"&rated=1&stats=1"
    start = time.time()# log the start time for this entry

    print(path)
    
    page = requests.get(path) # get the page
    game_page = BeautifulSoup(page.content, "xml") # parse the page with beautifulsoup
    
    
    try: 
        print(int(game_page.find('items')['totalitems']))
        
    except:
        
        print("failed, pausing")
        time.sleep(5)
        
        page = requests.get(path) # get the page
        game_page = BeautifulSoup(page.content, "xml") # parse the page with beautifulsoup
        print(int(game_page.find('items')['totalitems']))
    
    rated_items = game_page.find_all("item")
        
    game_ids = []
    game_ratings = []
    modified_record = []
    all_ratings = {}

    for game in rated_items:
        name = game.find("name").text
        game_id = game["objectid"] 
        rating = game.find("rating")["value"]
        date_rated = game.find("status")["lastmodified"]
        game_ids.append(game_id)
        game_ratings.append(rating)
        all_ratings[game_id]=rating
        modified = [username, game_id, rating, date_rated]
        modified_record.append(modified)

    raw_ratings_dict = {}
    raw_ratings_dict[username] = all_ratings
    
    user = pd.DataFrame.from_dict(raw_ratings_dict, orient="index")
    user_ratings_dates = pd.DataFrame(modified_record, columns=["Username", "BGGId", "Rating", "Date_rated"])
    
    time.sleep(.5)
    
    return user, user_ratings_dates
